# Collecting Financial Statements:
* SEC EDGAR Database: Extract urls to financial statements

In [ ]:
# @title Package Imports
import requests
import pandas as pd
import bs4 as bs
import numpy as np
from time import sleep
from concurrent.futures import ThreadPoolExecutor, as_completed
import json
import re

In [ ]:
# @title headers for interacting with SEC EDGAR DB
headers = {
    "User-Agent": "oarnst04@gmail.com"
}

## Get SEC identification information of all S&P 500 companies

In [ ]:
# @title extract_sp500_info
def extract_sp500_info():

  """
    Get ticker symbols of all S&P 500 companies

    Args:
        None

    Returns:
        sp500_info (pd.DataFrame) Ticker symbol, sector, and company name of all S&P 500 companies
  """

  sp500_info = pd.DataFrame(columns=["Ticker", "CompanyName", "Sector", "CompanyID"])

  # Use beautiful soup to parse html of wikipedia page for the table listing all S&P 500 companies
  response = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
  response.raise_for_status()
  soup = bs.BeautifulSoup(response.text, "lxml")
  table = soup.find('table', id="constituents")

  for row in table.findAll('tr')[1:]:
    cols = row.findAll('td')
    if len(cols) > 0:

        ticker = cols[0].text.strip()
        name = cols[1].text.strip()
        sector = cols[2].text.strip()
        cik = cols[6].text.strip()

        sp500_info.loc[len(sp500_info)] = [ticker, name, sector, cik]

  return sp500_info

In [ ]:
# @title test_SEC_info_retrieval
def test_SEC_info_retrieval():
    """
      Check if we retrieve all S&P 500 company tickers and extract a sufficient amount of cik strings

    Returns:
        sp500_SEC_ids (dict): cik strings for all companies in tickers
    """


    try:

      sp500_SEC = extract_sp500_info()
      sp500_SEC = sp500_SEC.drop_duplicates(subset="CompanyID", keep='first')
      sp500_SEC["CompanyID"] = sp500_SEC["CompanyID"].astype(int)

      return sp500_SEC

    except Exception as e:

      print("Failed")
      print(f"Error: {e}")

In [ ]:
sp500_SEC = test_SEC_info_retrieval()

## Retrieve URLs to 10-Q, 10-K, and 8-K forms for S&P 500 companies

In [ ]:
# @title parse_idx_file
def parse_idx_file(session, url, company_ids, year, quarter, form_types, extract_txt):

    """
      Parses the idx file to extract all 10-Q, 10-K, and 8-K forms. Helper function to fetch_filings_quarter.

      Args:
          session (requests.Session): Current session connection with SEC EDGAR DB
          url (str): url to idx file
          company_ids (list): cik string associated with each S&P 500 company
          year (int): the year in which we want the documents to be filed in
          quarter (int): the quarter in which we want the documents to be filed
          form_types (list): the form types to extract from the idx file
          extract_txt (bool): indicate extraction of .txt representation of financial statements

      Returns:
          df (pandas.DataFrame): Table containing the ticker symbol, cik string, form type, year,
                                  quarter, and url for each document filed in the specified year and quarter.
    """

    df = pd.DataFrame(columns=["Ticker", "CompanyID", "Form Type", "Year", "Quarter", "URLs"])

    response = session.get(url)
    response.raise_for_status()

    lines = response.text.splitlines()
    data_lines = lines[10:]

    for line in data_lines:

        data = line.split()
        curr_cik = int(str(data[-3]))
        form_type = str(data[-4])

        # Check if current row contains information about the wanted form types
        if (curr_cik in company_ids) and (form_type in form_types):

            # extract url to directory
            if not extract_txt:
                file_path = data[-1].replace("-", "").rstrip(".txt")
            # extract url to .txt file
            else:
                file_path = data[-1]

            document = f"https://www.sec.gov/Archives/{file_path}"

            df.loc[len(df)] = [company_ids[curr_cik], curr_cik, form_type, year, quarter, document]

    return df

In [ ]:
# @title fetch_filings and fetch_filings_quarter
def fetch_filings(headers, company_ids, year, form_types, extract_txt):

    """
      Retrieve URLs to all company filings (specified by form_types) for a given year

      Args:
          headers (dict): Required headers to access SEC EDGAR DB API
          company_ids (list): cik strings associated with each company in the S&P 500
          year (int): the year in which we want the documents to be filed in
          form_types (list): the form types to extract from the idx file
          extract_txt (bool): indicate extraction of .txt representation of financial statements

      Returns:
          filings_df (pandas.DataFrame): Table containing the ticker symbol, cik string, form type, year,
                                         quarter, and url for each document filed in the specified year

    """

    # Open a connection to SEC EDGAR DB
    session = requests.Session()
    session.headers.update(headers)

    def fetch_filings_quarter(quarter, year=year):

      """
        Retrieve URLs to all 10-K, 10-Q, and 8-K forms of all S&P 500 companies for a specific year and quarter.
        Helper function to parallelize fetch_filings.

        Args:
            year (int): the year in which we want the documents to be filed in
            quarter (int): the quarter in which we want the documents to be filed


        Returns:
            filings_df (pandas.DataFrame): Table containing the ticker symbol, cik string, form type, year,
                                           quarter, and url for each document filed in the specified year and quarter
      """

      idx_url = f"https://www.sec.gov/Archives/edgar/full-index/{year}/QTR{quarter}/company.idx"

      try:
        filings_df = parse_idx_file(session, idx_url, company_ids, year, quarter, form_types=form_types, extract_txt=extract_txt)
      except Exception as e:
        print(f"An error occurred: {e}")
        return

      print(f"Finished fetching filings for quarter {quarter}")
      sleep(0.5)

      return filings_df


    # Use ThreadPoolExecutor to run fetch_for_quarter in parallel for each quarter
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(fetch_filings_quarter, quarter) for quarter in range(1, 5)]
        filings_results = []
        for future in as_completed(futures):
            result = future.result()
            if result is not None:
                filings_results.append(result)

    # close connection
    session.close()

    # Combine dataframes for each quarter
    filings_df = pd.concat(filings_results, ignore_index=True)

    return filings_df

In [ ]:
# @title test_filings_url_extraction
def test_filing_url_extraction(headers, sp500_SEC, year, form_types, extract_txt=False):

    """
      Makes sure fetch_filings doesn't raise an error

      Args:
          headers (dict): Required headers to access SEC EDGAR DB API
          sp500_SEC (dict): cik information associated with each company in S&P 500 index
          year (int): the year in which we want the documents to be filed in
          form_types (list): the form types to extract
          extract_txt (bool): indicate extraction of .txt representation of financial statements

      Returns:
          filings_df (pandas.DataFrame): Table containing the ticker symbol, cik string, form type, year,
                                         quarter, and url for each document filed in the specified year
    """

    try:

      company_info = dict(zip(sp500_SEC["CompanyID"].values, sp500_SEC["Ticker"].values))
      filings_df = fetch_filings(headers, company_info, year, form_types, extract_txt=extract_txt)

      print("Passed")

      return filings_df

    except Exception as e:

      print("Failed")
      print(f"Error: {e}")

      return

In [ ]:
# @title Ex: Extract urls to 10-Q and 10-K filings of S&P 500 companies from 2016-2024
quarterly_and_annual_filings-df = pd.DataFrame()
for i in range(2016, 2025):
    quarterly_and_annual_filings-df = pd.concat([quarterly_and_annual_filings-df, test_filing_url_extraction(headers, sp500_SEC, i, ["10-K", "10-Q"], extract_txt=False)], ignore_index=True)
quarterly_and_annual_filings-df.sort_values(by=["Ticker", "Year", "Quarter"], inplace=True)

In [ ]:
# @title Ex: Extract links to .txt representation of 2024 10-K forms
tenK_filings = pd.DataFrame()
for i in range(2023, 2025):
    tenK_filings = pd.concat([tenK_filings, test_filing_url_extraction(headers, sp500_SEC, i, ["10-K"], extract_txt=True)], ignore_index=True)
tenK_filings.sort_values(by=["Ticker", "Year"], inplace=True)
tenK_filings = tenK_filings[~tenK_filings["Ticker"].duplicated(keep="last")]